In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from synorchestrator import orchestrator
from synorchestrator.trs.client import TRSClient
from synorchestrator.wes.client import WESClient
from synorchestrator import config

In [3]:
config.show()


Orchestrator options:

Evaluation Queues
---------------------------------------------------------------------------
wflow0: github.com/dockstore-testing/md5sum-checker [CWL]
wflow1: github.com/dockstore-testing/md5sum-checker/wdl [WDL]
wflow2: github.com/DataBiosphere/topmed-workflows/TopMed_Variant_Caller [WDL]
wflow3: github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline [WDL]

Tool Registries
---------------------------------------------------------------------------
dockstore: dockstore.org:8443

Workflow Services
---------------------------------------------------------------------------
hca-cromwell: g0n2qjnu94.execute-api.us-east-1.amazonaws.com/test
arvados-wes: wes.qr1hi.arvadosapi.com
local: 0.0.0.0:8080


In [4]:
orchestrator.run_all({
    'wflow2': ['hca-cromwell'],
    'wflow3': ['hca-cromwell']
})

INFO:synorchestrator.orchestrator:Preparing checker workflow run request for 'github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline' from  'dockstore''
INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow3 (github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline)
 - submission ID: '250525170506017965'
 - data:
{
  "workflow_descriptor": "import \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.13.0/aligner/u_of_michigan_aligner/u_of_michigan_aligner.wdl\" as TopMed_aligner\nimport \"https://raw.githubusercontent.com/DataBiosphere/topmed-workflows/1.13.0/aligner/functional-equivalence-checker/topmed-alignment-checker.wdl\" as checker\n\nworkflow checkerWorkflow {\n  Int expectedNumofReads\n  String docker_image\n\n  File input_crai_file\n  File input_cram_file\n\n  File ref_alt\n  File ref_bwt\n  File ref_sa\n  File ref_amb\n  File ref_ann\n  File ref_pac\n\n  File ref_fasta\n  File ref_fasta_index\n\

INFO:synorchestrator.orchestrator:Submitting job '250525170506017965' for eval 'wflow3' to WES endpoint 'hca-cromwell'
INFO:synorchestrator.orchestrator:Submitting job '250525170508043137' for eval 'wflow2' to WES endpoint 'hca-cromwell'


KeyError: 'workflow_id'

In [ ]:
orchestrator.monitor(['wflow2', 'wflow3'])

Time elapsed: 0h:4m:56s



,,wes,job,run_status,run_id
TopMed_Variant_Caller,250525170508043137,hca-cromwell,checker,COMPLETE,48c766f4-3d83-45da-9ba3-8f1d5a6ab12f
u_of_Michigan_alignment_pipeline,250525170506017965,hca-cromwell,checker,RUNNING,2a655b12-3007-433c-b7a1-99a7d2d4e945
